## Load Data
directly loading from submission 2

In [1]:
import pandas as pd

train_path = "output_2/train_data.pkl"
test_path = "output_2/test_data.pkl"

#載入保存的 pickle 文件
train_data = pd.read_pickle(train_path)
test_data = pd.read_pickle(test_path)

#train_data
test_data

,tweet_id,text,hashtags,crawldate,identification,emotion
2,0x28b412,"Confident of your obedience, I write to you, k...",[bibleverse],2017-12-25 04:39:20,test,NaN
4,0x2de201,"""Trust is not the same as faith. A friend is s...",[],2016-01-08 17:18:59,test,NaN
9,0x218443,When do you have enough ? When are you satisfi...,"[materialism, money, possessions]",2015-09-09 09:22:55,test,NaN
30,0x2939d5,"God woke you up, now chase the day #GodsPlan #...","[GodsPlan, GodsWork]",2015-10-10 14:33:26,test,NaN
33,0x26289a,"In these tough times, who do YOU turn to as yo...",[],2016-10-23 08:49:50,test,NaN
...,...,...,...,...,...,...
1867525,0x2913b4,"""For this is the message that ye heard from th...",[],2016-12-10 18:01:00,test,NaN
1867529,0x2a980e,"""There is a lad here, which hath five barley l...",[],2015-01-04 14:40:55,test,NaN
1867530,0x316b80,When you buy the last 2 tickets remaining for ...,"[mixedfeeling, butimTHATperson]",2015-05-12 12:51:52,test,NaN
1867531,0x29d0cb,I swear all this hard work gone pay off one da...,[],2017-10-02 17:54:04,test,NaN


## preprocessing

### BOW

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_500.fit(train_data['text'])

train_data_BOW_features_500 = BOW_500.transform(train_data['text'])
#test_data_BOW_features_500 = BOW_500.transform(test_data['text'])

## check dimension
train_data_BOW_features_500.shape
#test_data_BOW_features_500.shape

ModuleNotFoundError: No module named 'nltk'

### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 初始化 TfidfVectorizer，并限制特征数为 1000
tfidf_vect = TfidfVectorizer(max_features=1000)

# 使用 TF-IDF 向量化器进行训练集和测试集的向量化
X_train = tfidf_vect.fit_transform(train_data["text"])
X_test = tfidf_vect.transform(test_data["text"])

# 获取特征名称列表
feature_names = tfidf_vect.get_feature_names_out()

# 显示索引 100 到 110 的特征名称
selected_features = feature_names[100:110]
print("Features from index 100 to 110:", selected_features)
train_tfidf

Features from index 100 to 110: ['asked' 'asking' 'ass' 'at' 'august' 'away' 'awesome' 'awkward' 'baby'
 'back']


<1455563x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 13350564 stored elements in Compressed Sparse Row format>

In [ ]:
# for a classificaiton problem, you need to provide both training & testing data
# 假設 train_data 包含特徵欄位 X 和目標欄位 y
y_train = train_data['emotion']
y_test = test_data['emotion']



In [ ]:
import os
import pickle

# 設定儲存資料夾路徑
output_folder = 'output_3'
os.makedirs(output_folder, exist_ok=True)  # 如果資料夾不存在，則創建

# 轉換數據
X_train = BOW_500.transform(train_data['text'])
y_train = train_data['emotion']

X_test = BOW_500.transform(test_data['text'])
y_test = test_data['emotion']

# 保存到指定資料夾
with open(os.path.join(output_folder, 'X_train.pkl'), 'wb') as f:
    pickle.dump(X_train, f)

with open(os.path.join(output_folder, 'y_train.pkl'), 'wb') as f:
    pickle.dump(y_train, f)

with open(os.path.join(output_folder, 'X_test.pkl'), 'wb') as f:
    pickle.dump(X_test, f)

with open(os.path.join(output_folder, 'y_test.pkl'), 'wb') as f:
    pickle.dump(y_test, f)

print(f"Data saved to '{output_folder}' folder as .pkl files")

Data saved to 'output_3' folder as .pkl files


## pkl files

In [2]:
import pandas as pd

# 讀取 .pkl 文件
X_train = pd.read_pickle('output_3/X_train.pkl')
y_train = pd.read_pickle('output_3/y_train.pkl')
X_test = pd.read_pickle('output_3/X_test.pkl')
y_test = pd.read_pickle('output_3/y_test.pkl')

train_path ='output_1/train_data.pkl'
test_path ='output_1/test_data.pkl'
train_data = pd.read_pickle(train_path)
test_data = pd.read_pickle(test_path)

# 檢查數據
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (1455563, 500)
y_train.shape:  (1455563,)
X_test.shape:  (411972, 500)
y_test.shape:  (411972,)


## CSV Logger (with hidden layer)

In [ ]:
from sklearn.preprocessing import LabelEncoder
import keras

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train:\n', y_train)
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
#y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train:\n', y_train)
print('\ny_train.shape: ', y_train.shape)
#print('y_test.shape: ', y_test.shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train:
 0          anticipation
1               sadness
3                  fear
5                   joy
6          anticipation
               ...     
1867526             joy
1867527             joy
1867528             joy
1867533             joy
1867534             joy
Name: emotion, Length: 1455563, dtype: object

y_train.shape:  (1455563,)
y_test.shape:  (411972,)


## After convert
y_train:
 [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

y_train.shape:  (1455563, 8)


### build model

In [ ]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  1000
output_shape:  8


In [ ]:
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  
X = model_input

# 1st hidden layer
X_W1 = Dense(units=32)(X)  # Reduced units
H1 = ReLU()(X_W1)

# output layer
H1_W2 = Dense(units=output_shape)(H1)  # Output units
H2 = Softmax()(H1_W2)

model_output = H2

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

input_shape:  1000
output_shape:  8


2024-11-14 20:35:43.240439: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-14 20:35:43.240670: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1000)]            0         
                                                                 
 dense (Dense)               (None, 32)                32032     
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 264       
                                                                 
 softmax (Softmax)           (None, 8)                 0         
                                                                 
Total params: 32,296
Trainable params: 32,296
Non-trainable params: 0
_________________________________________________________________


### train

In [13]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 100
batch_size = 16

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test, y_test))
print('training finish')

2024-11-14 20:35:49.429154: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at serialize_sparse_op.cc:389 : INVALID_ARGUMENT: indices[2] = [0,675] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.




InvalidArgumentError: {{function_node __wrapped__SerializeManySparse_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[2] = [0,675] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse]

### Predict on testing data

In [ ]:
## predict
pred_result = model.predict(X_test, batch_size=128)
y_pred = label_decode(label_encoder, pred_result)



## Voting做比例合成

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score


# 初始化 LightGBM 和 RandomForest 模型
lgbm = LGBMClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)

# 創建 Voting Classifier，將兩個模型的輸出結果進行投票
voting_clf = VotingClassifier(estimators=[
    ('lgbm', lgbm), ('rf', rf)
], voting='soft')  # 使用 'soft' 投票方式以概率加權

# 訓練 Voting 模型
voting_clf.fit(X_train, y_train)

# 預測並計算準確率
y_pred = voting_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Voting Classifier Accuracy: {accuracy * 100:.2f}%")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.987067 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2310
[LightGBM] [Info] Number of data points in the train set: 1455563, number of used features: 500
[LightGBM] [Info] Start training from score -3.597599
[LightGBM] [Info] Start training from score -1.765956
[LightGBM] [Info] Start training from score -2.347948
[LightGBM] [Info] Start training from score -3.124281
[LightGBM] [Info] Start training from score -1.037008
[LightGBM] [Info] Start training from score -2.018196
[LightGBM] [Info] Start training from score -3.396874
[LightGBM] [Info] Start training from score -1.957809
